### Retail Sales and Inventories

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### Main Chart

In [2]:
# Retail sales
url = ('https://api.census.gov/data/timeseries/eits/marts?'+
       'get=cell_value,time_slot_id,category_code&'+
       f'key={census_key}&data_type_code=SM&time=from+1992&'+
       'for=us&seasonally_adj=yes')
r = requests.get(url).json()
date = lambda x: pd.to_datetime(x.time)
df = ((pd.DataFrame(r[1:], columns=r[0]).assign(date = date)
         .set_index(['date', 'category_code']))['cell_value']
         .unstack().astype('float'))
print('Latest: ', dtxt(df.index[-1])['mon1'])
df.to_csv(data_dir / 'rs_raw.csv', index_label='date')

Latest:  September 2023


In [3]:
df = pd.read_csv(data_dir / 'rs_raw.csv', index_col='date', 
                 parse_dates=True)[['44000', '44X72', '44W72', '454']]

# Retail sales growth
data = (df.pct_change(12) * 100).dropna()
res = data[['44X72', '454']].rolling(3).mean().dropna()
res.to_csv(data_dir / 'marts.csv', index_label='date', 
            float_format='%g')
adj = node_adj(res)
smax = res.iloc[-1].idxmax()
adj[smax] = adj[smax] + 0.35

colors = {'44X72': 'green!70!black', '454': 'blue!60!black'}
date = {series: 'm' if series == smax else None 
        for series in colors.keys()}
nodes  ='\n'.join([end_node(res[series], color, 
                            date=date[series], 
                            size=1.1, offset=adj[series]) 
                   for series, color in colors.items()])
write_txt(text_dir / 'rs_nodes.txt', nodes)  
ltdt = dtxt(df.index[-1])['mon1']
lttot = value_text(data['44X72'].iloc[-1], 'plain')
lt3m = value_text(res['44X72'].iloc[-1], 'plain')
ns3m = value_text(res['454'].iloc[-1], 'plain')
text = ('Changes in retail and food services sales can '+
        'indicate shifts in consumer behavior. One-year '+
        'retail and food services \\textbf{sales growth} '+
        f'is {lttot} in {ltdt}, and averages {lt3m} '+
        f'over past three months {c_line(colors["44X72"])}. '+
        'Nonstore sales, for example from online '+
        'retailers, have increased at a '+
        'faster rate than other sales, since 1992. '+
        'Over the past three months, one-year nonstore '+
        f'sales growth averages {ns3m} '+
        f'{c_line(colors["454"])}. ')
write_txt(text_dir / 'rs_gr.txt', text)
print(text, '\n')

# Share of aggregate measures
gdpm = pd.read_csv(data_dir / 'gdp_monthly.csv', index_col='date', 
                   parse_dates=True)['A191RC']

dpi = (pd.read_csv(data_dir / 'nipa20600.csv', 
                   index_col='date', parse_dates=True)
         .loc['1992':, 'A067RC'].rename('VALUE'))

pop = (pd.read_csv(data_dir/'nipa20600.csv', index_col='date', 
                  parse_dates=True)['B230RC'])
pce = (pd.read_csv(data_dir / 'pi_raw.csv', index_col='date', 
                  parse_dates=True)['DPCERC'])

pi = (pd.read_csv(data_dir / 'nipa20804.csv', 
                 index_col='date', parse_dates=True)['DPCERG'])
pch = (pi.iloc[-1] / pi)

rs = df['44X72']
sh = pd.DataFrame()
sh['DPI'] = (rs.multiply(12).divide(dpi, axis=0) * 100).dropna()
sh['CE'] = (rs.multiply(12).divide(pce, axis=0) * 100).dropna()
sh['GDP'] = (rs.multiply(12).divide(gdpm, axis=0) * 100).dropna()
sh['RealPC_Tot'] = (rs.multiply(pch, axis=0)
               .dropna().divide(pop / 1000, axis=0)).dropna()
sh['RealPC_ExAutoGas'] = (df['44W72'].multiply(pch, axis=0)
               .dropna().divide(pop / 1000, axis=0)).dropna()
sh['RealPC_Nonstore'] = (df['454'].multiply(pch, axis=0)
               .dropna().divide(pop / 1000, axis=0)).dropna()
sh.to_csv(data_dir / 'rs_sh.csv', index_label='date')

rsdt = dtxt(sh['RealPC_Tot'].dropna().index[-1])['mon1']
write_txt(text_dir / 'rs_dt.txt', rsdt)
adj = node_adj(sh[['DPI', 'CE', 'GDP']])
smax = sh[['DPI', 'CE', 'GDP']].iloc[-1].idxmax()
adj[smax] = adj[smax] + 0.35

colors = {'DPI': 'magenta!80!purple', 'GDP': 'blue', 'CE': 'cyan'}
date = {series: 'm' if series == smax else None 
        for series in colors.keys()}
nodes  ='\n'.join([end_node(sh[series], color, 
                            date=date[series], 
                            size=1.1, offset=adj[series]) 
                   for series, color in colors.items()])
write_txt(text_dir / 'rs_sh_nodes.txt', nodes)  

ltdt = dtxt(sh.index[-1])['mon1']
tot = df['44X72'].iloc[-1] / 1_000
lt = sh.iloc[-1]
text = (f'In {ltdt}, \\textbf{{retail and food services sales}} '+
        f'total \${tot:.1f} billion. On an annualized basis, '+
        f'this is equivalent to {lt.DPI:.1f} percent of '+
        f'disposable (after-tax) income {c_line(colors["DPI"])}, '+
        f'{lt.CE:.1f} percent of consumer spending '+
        f'{c_line(colors["CE"])}, and {lt.GDP:.1f} percent of '+
        f'GDP {c_line(colors["GDP"])}.')
write_txt(text_dir / 'rs_sh.txt', text)
print(text, '\n')

# Real per capita retail sales
colors = {'RealPC_Tot': 'violet!60!blue', 
          'RealPC_ExAutoGas': 'green!80!olive',
          'RealPC_Nonstore': 'blue!60!black'}
adj = node_adj(sh[colors.keys()])
smax = sh[colors.keys()].iloc[-1].idxmax()

date = {series: 'm' if series == smax else None 
        for series in colors.keys()}
nodes  ='\n'.join([end_node(sh[series], color, 
                            date=date[series], dollar=True,
                            digits='comma',
                            size=1.1, offset=adj[series]) 
                   for series, color in colors.items()])
write_txt(text_dir / 'rs_pc_nodes.txt', nodes)  

pclt = lt['RealPC_Tot']
pc19 = sh.loc['2019', 'RealPC_Tot'].mean()
pcex = lt['RealPC_ExAutoGas']
pcex19 = sh.loc['2019', 'RealPC_ExAutoGas'].mean()

text = ('Per capita retail and food services sales, '+
        'adjusted by the personal consumption expenditure '+
        f'(PCE) price index, are \${pclt:,.0f} during {ltdt} '+
        f'{c_line(colors["RealPC_Tot"])}. Prior to the '+
        'pandemic, in 2019, real per capita retail and '+
        f'food service sales averaged \${pc19:,.0f} per '+
        'month. Excluding automotive and gasoline sales, '+
        f'per capita sales were \${pcex:,.0f} in {ltdt} '+
        f'and \${pcex19:,.0f} per month in 2019, after '+
        'adjusting for inflation '+
        f'{c_line(colors["RealPC_ExAutoGas"])}. ')
write_txt(text_dir / 'rs_pc.txt', text)
print(text)

Changes in retail and food services sales can indicate shifts in consumer behavior. One-year retail and food services \textbf{sales growth} is 3.8 percent in September 2023, and averages 3.1 percent over past three months (see {\color{green!70!black}\textbf{---}}). Nonstore sales, for example from online retailers, have increased at a faster rate than other sales, since 1992. Over the past three months, one-year nonstore sales growth averages 8.4 percent (see {\color{blue!60!black}\textbf{---}}).  

In September 2023, \textbf{retail and food services sales} total \$704.9 billion. On an annualized basis, this is equivalent to 41.6 percent of disposable (after-tax) income (see {\color{magenta!80!purple}\textbf{---}}), 44.9 percent of consumer spending (see {\color{cyan}\textbf{---}}), and 30.0 percent of GDP (see {\color{blue}\textbf{---}}). 

Per capita retail and food services sales, adjusted by the personal consumption expenditure (PCE) price index, are \$2,100 during September 2023 (

### By Kind of Business

In [4]:
cats = [('441', 'Motor Vehicles \& Parts', 'blue!70!cyan'), 
        ('442', 'Furniture \& Home Furnishings', 'orange!90!red'), 
        ('443', 'Electronics \& Appliance', 'red!80!orange!90!white'), 
        ('444', 'Building \& Garden Equipment', 'blue!70!black'), 
        ('445', 'Food \& Beverage Stores', 'green!50!black'), 
        ('446', 'Health \& Personal Care', 'green!80!blue!90!white'), 
        ('447', 'Gasoline Stations', 'pink!80!red'), 
        ('448', 'Clothing and Accessories', 'yellow!75!orange'), 
        ('451', 'Sports/Hobby/Music/Books', 'violet'), 
        ('452', 'General Merchandise', 'green!60!yellow'), 
        ('454', 'Nonstore', 'purple!90!black'), 
        ('722', 'Food Service \& Drinking Places', 'cyan!90!white')]

names = {i[0]: i[1] for i in cats}
df = pd.read_csv(data_dir / 'rs_raw.csv', index_col='date', 
                 parse_dates=True)[names.keys()]
    
dpi = (pd.read_csv(data_dir / 'nipa20600.csv', 
                   index_col='date', parse_dates=True)
         .loc['1992':, 'A067RC'].rename('VALUE'))
data = (df.multiply(12).divide(dpi, axis=0) * 100).dropna()

data.to_csv(data_dir / 'rs_ind.csv', index_label='date')

maxdt = dtxt(data.index[-1] + pd.DateOffset(weeks = 10))['datetime']

d = {}
for sname, name, color in cats:
    d[f'{sname}_line'] = f'''\\rebars
                \stdline{{{color}}}{{date}}{{{sname}}}{{data/rs_ind.csv}}
                \\node[text width=4.2cm, anchor=west] at 
                    (axis description cs: -0.24, 1.11){{\\footnotesize{{{name}}}}};
                {end_node(data[sname], color)}'''
    mean = round(data[sname].mean(), 1)
    ytick = ', '.join([str(round(data[sname].max(), 1)), str(mean),
                     str(round(data[sname].min(), 1))])
    d[f'{sname}_end'] = f'''\shdateaxisticks  
		yticklabel style={{text width=1.6em}}, enlarge y limits={{0.08}},
        ytick={{{ytick}}}, \\bbar{{y}}{{{mean}}}
		xticklabel={{`\short{{\year}}}}, clip=false, height=3.3cm, width=4.6cm
    '''
    d[sname] = f'''date coordinates in=x, axis line style={{draw=none}},
    	xticklabels=\empty, xtick style={{draw=none}},
		xmax={{{maxdt}}}, max space between ticks=40,	    
		enlarge y limits={{0.08}},  \\bbar{{y}}{{{mean}}}
		enlarge x limits={{0.01}}, ytick={{{ytick}}}, 
		yticklabel style={{text width=1.6em}},
		clip=false, height=3.3cm, width=4.6cm'''

lt = data.iloc[-1].sort_values(ascending=False)
top9 = '\n'.join([f'\\nextgroupplot[{d[s]}] {d[f"{s}_line"]}' 
                  for s in lt[:9].index])
bottom3 = '\n'.join([f'\\nextgroupplot[{d[f"{s}_end"]}] {d[f"{s}_line"]}' 
                     for s in lt[9:].index])    
    
v = (f'''\\begin{{groupplot}}[group style={{group size=3 by 4, 
    x descriptions at=edge bottom, vertical sep=20pt, horizontal sep=36pt,}},]
    {top9}
    {bottom3}
    \end{{groupplot}}''')
write_txt(text_dir / 'rs_ind.txt', v)

ns92 = data['454'].iloc[0]
nslt = data['454'].iloc[-1]
ltdt = dtxt(data.index[-1])['mon1']
nssh = dpi.iloc[-1] * ((nslt - ns92) / 100) / 1_000
ch92 = (data.iloc[-1] - data.iloc[0]).sort_values()
endings = ['store', 'stations', 'places']
tv = {}
for i in [0, 1, 2]:
    cat = names[ch92.index[i]].lower().replace('\\&', 'and')
    if any(ending in cat for ending in endings) == False:
        cat = cat + ' stores'
    tv[i] = f'{cat} ({ch92.iloc[i]:.1f} percentage points)'
text = (f'Nonstore sales were {ns92:.1f} percent of '+
        f'after-tax income in January 1992 and {nslt:.1f} percent '+
        f'in {ltdt}, a shift that is equivalent to \${nssh:,.0f} '+
        'billion per year. Since 1992, sales '+
        f'as a share of after-tax income has decreased in {tv[0]}, '+
        f'{ tv[1]}, and {tv[2]}.')
write_txt(text_dir / 'rs_kb.txt', text)
print(text)

rsdt = f'January 1992 to {ltdt}'
write_txt(text_dir / 'rs_dates.txt', rsdt)

Nonstore sales were 1.7 percent of after-tax income in January 1992 and 7.0 percent in September 2023, a shift that is equivalent to \$1,077 billion per year. Since 1992, sales as a share of after-tax income has decreased in food and beverage stores (-2.9 percentage points), clothing and accessories stores (-1.0 percentage points), and general merchandise stores (-0.7 percentage points).


### Inventories to Sales Ratio (ISRATIO)

In [5]:
# Nominal
url = ('https://api.census.gov/data/timeseries/eits/mtis?'+
       'get=cell_value,time_slot_id,category_code&'+
       f'key={census_key}&data_type_code=IR&category_code=TOTBUS&'+
       'time=from+1992&for=us&seasonally_adj=yes')
r = requests.get(url).json()
date = lambda x: pd.to_datetime(x.time)
df = (pd.DataFrame(r[1:], columns=r[0]).assign(date = date)
        .set_index('date'))['cell_value'].astype('float').sort_index()
df.name = 'VALUE'
df.to_csv(data_dir / 'isratio.csv', index_label='date')

# Real 
years = ','.join(map(str, range(1988, 2024)))
url = (f'https://apps.bea.gov/api/data/?&UserID={bea_key}'+
        '&method=GetData&datasetname=NIUnderlyingDetail&'+
        'TableName=U003BU&LineNumber=1&Frequency=M&'+
       f'Year={years}&ResultFormat=json')
r = requests.get(url).json()
data = r['BEAAPI']['Results']['Data']

s1 = pd.read_csv('raw/real_isratio.csv')
s1['Date'] = pd.to_datetime(s1['Date'])
s1 = s1.set_index('Date')['Value'].sort_index()

s2 = pd.Series({pd.to_datetime(i['TimePeriod'], format='%YM%m'): 
               i['DataValue'] for i in data 
                if i['LineNumber'] == '1'}).astype(float)
s2.index.name = 'Date'
s2.name = 'Value'
s2 = s2.sort_index()
result = pd.concat([s1, s2])
result.to_csv(data_dir / 'real_isratio.csv', index_label='date')

# Text
ltmon = dtxt(df.index[-1])['mon1']
prmon = dtxt(df.index[-2])['mon1']
prmon12 = dtxt(df.index[-13])['mon1']
ltval = f'{df.iloc[-1]:.2f}'
prval = f'{df.iloc[-2]:.2f}'
prval12 = f'{df.iloc[-13]:.2f}'
pcval = f'{df.loc["2020-02-01"].mean():.2f}'
lt2mon = dtxt(result.index[-1])['mon1']
pr2mon = dtxt(result.index[-13])['mon1']
lt2val = f'{result.iloc[-1]:.2f}'
lt2val2 = f'{result.iloc[-13]:.2f}'
rngval = f'{result.loc["2019"].mean():.2f}'

text = (f'In {ltmon}, the ratio of total business '+
        f'inventories to sales was {ltval}, compared '+
        f'to {prval} in {prmon}, {prval12} in {prmon12}, '+
        f'and {pcval} in February 2020.\n\nThe '+
        'inflation-adjusted version from BEA shows '+
        f'inventories at {lt2val} times sales in '+
        f'{lt2mon}, following a ratio of {result.iloc[-2]:.2f} '+
        f'in {dtxt(result.index[-2])["mon1"]}, '+
        f'and {lt2val2} one year prior, in {pr2mon}. '+
        f'In 2019, real monthly inventories were {rngval} '+
         'times real monthly sales, on average. ')
write_txt(text_dir / 'isratio.txt', text)
print(text)

In August 2023, the ratio of total business inventories to sales was 1.37, compared to 1.39 in July 2023, 1.36 in August 2022, and 1.43 in February 2020.

The inflation-adjusted version from BEA shows inventories at 1.55 times sales in August 2023, following a ratio of 1.55 in July 2023, and 1.54 one year prior, in August 2022. In 2019, real monthly inventories were 1.55 times real monthly sales, on average. 
